In [2]:
from google.colab import drive
drive.mount('/content/drive')
import csv
import re
import random
import tqdm.notebook as tq
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/threads/cht-2020-merged.txt'

with open(file_path, 'r', encoding = 'utf-8') as f:
  comments = [ line.strip() for line in f ]

In [9]:
construcion_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/constructions_0331.csv'

with open(construcion_path) as f:
    constructions = [ row.strip() for row in f ]

In [10]:
constructions

['只有遠傳\\s?沒有\\w{2}',
 '純噓\\w+\\b',
 '\\b\\w+能\\w\\s?，?\\w+都能\\w了\\b',
 '八成\\w+\\b',
 '\\b\\w+秀下限\\w+\\b',
 '\\b誰敢\\w+我一個一個\\w出來\\b',
 '勿忘\\w+\\b',
 '\\b\\w+心酸的?\\w*\\b',
 '\\b\\w+根本盤子?\\b',
 '\\b\\w+敲盤子\\w*\\b',
 '\\b\\w+被當盤子\\w*\\b',
 '普遍\\w+\\b',
 '\\b\\w+給力\\b',
 '\\b\\w+屌打\\w+\\b',
 '敲碗等\\w+\\b',
 '(置|拉)板凳等\\w+\\b',
 '坐等\\w+\\b',
 '\\b\\w+(的|得)要死',
 '\\b\\w到爆\\w?\\b',
 '\\b三不五時就\\w+\\b',
 '(真的)?有夠\\w+的?\\b',
 '也太\\w+了(吧)?',
 '直接\\w+(了)?\\b',
 '\\w到(快|要)?吐血\\b',
 '對\\w+一點(都|也)不\\w+\\b',
 '誰\\w誰\\w+\\b',
 '被\\w+到死',
 '我也是\\w+啦',
 '\\b\\w+(掰|bye)了',
 '\\b\\w+到\\w*炸了?',
 '\\b\\w+是在哈囉',
 '真(的|ㄉ)\\w+到有剩',
 '越來越\\w\\b',
 '一點(都|也)不\\w+\\b',
 '\\b\\w+廢到笑',
 '\\b\\w+(到|的|得)離譜',
 '\\b\\w+非洲',
 '沒一個\\w+的',
 '\\b\\w+雞肋',
 '輸\\w+好幾年',
 '很可以\\w*',
 '真的有\\w+到\\b',
 '\\w+就贏了',
 '(很|也)夠\\w+了']

In [11]:
n = 5  # sample size
random.seed(10)

matched = {}
for k in tq.tqdm(constructions):
  pat = re.compile(k)
  if k not in matched: matched[k] = []
  
  # Get all cnstr from all comments
  candidates = set()
  for cmt in comments:
    for c in pat.finditer(cmt): candidates.add(c[0])
  
  # Sample
  if len(candidates) > n:
    for c in random.sample(candidates, n): matched[k].append(c)
  else:
    for c in candidates: matched[k].append(c)

In [14]:
import pprint
#pprint.pprint(matched)

In [13]:
df = pd.DataFrame([matched], columns=matched.keys()).T
df.columns = ['examples']
df

,examples
只有遠傳\s?沒有\w{2},"[只有遠傳 沒有良心, 只有遠傳 沒有距離, 只有遠傳 沒有速度, 只有遠傳沒有網路, 只有..."
純噓\w+\b,"[純噓樓上, 純噓東東, 純噓遠傳, 純噓中華行銷叨擾長輩, 純噓台星5g]"
\b\w+能\w\s?，?\w+都能\w了\b,"[店員的話能信，連屎都能吃了, 屁話能信史都能吃了]"
八成\w+\b,"[八成給遠傳了都要等不如訂官方商城要現貨, 八成又是台灣之星, 八成的攤位都是與教育有關的教..."
\b\w+秀下限\w+\b,"[別出來秀下限了, 厚顏無恥集體秀下限的國家特許電信商, 不懂別秀下限了, 一定會有貪小便宜..."
\b誰敢\w+我一個一個\w出來\b,[誰敢辦台星我一個一個抓出來]
勿忘\w+\b,"[勿忘499之亂, 勿忘iphone, 勿忘黑心頂新, 勿忘記亞太是以電信服務為主的公司]"
\b\w+心酸的?\w*\b,"[換5g真的是換心酸的, 網路費好像繳心酸的一樣, 賣東西不獲利是作心酸的嗎沒空間就只能找有..."
\b\w+根本盤子?\b,"[費300根本盤, 5m根本盤, 用5g根本盤子, 絕對不要續約根本盤子, 綁四年根本盤]"
\b\w+敲盤子\w*\b,"[蘋果又出來敲盤子, 繼續敲盤子, 根本敲盤子, 不然台灣人還在被三大敲盤子, 各家都拿高資..."
